# **Launch Sites Locations Analysis with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
#install libraries
!pip install folium
!pip install pandas
print('***all packages installed***'.upper())

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
***ALL PACKAGES INSTALLED***


In [2]:
import folium
import pandas as pd
# Import MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium Mouseposition plugin
from folium.plugins import MousePosition
# Import folium Devicon plugin
from folium.features import DivIcon

## Mark all launch sites on a map

In [3]:
#get the dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is Cape Canaveral in Florida
cape_canaveral = [28.562302,-80.577356]
site_map = folium.Map(location=cape_canaveral, zoom_start=10)
site_map

Create and add folium.Circle and folium.Marker for each launch site on the site map

In [6]:
cape_canaveral = [28.562302,-80.577356]
site_map = folium.Map(location=cape_canaveral, zoom_start=5)
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=50, 
        color='#d35400', 
        fill=True).add_child(
            folium.Popup(site['Launch Site']))
    marker = folium.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
            )    
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# Mark the success/failed launches for each site on the map

In [7]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [8]:
#create a Markercluster object
marker_cluster = MarkerCluster()

In [9]:
#create a new column name 'marker_color' if class is 1 then 'green' else 'red' 
marker_color =[]
for outcome in spacex_df['class']:
    if outcome == 1:
        marker_color.append('green')
    else:
        marker_color.append('red')
spacex_df['marker_color'] = marker_color
spacex_df.tail()

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


### Create icons of launch sites, if the launch site was a sucess then icon should be the color green with a popup of 'Success' else icon should be the color red and with a popup of 'Fail'

In [10]:
# add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, site in spacex_df.iterrows():
    # add a color 
    icon_color = 'green' if site['class'] == 1 else 'red'
    if site['marker_color'] == 'green':
        popup ='Success'
    else: 
        popup= 'Fail'
    marker= folium.Marker(
        location=[site['Lat'], site['Long']],
        icon=folium.Icon(color='white', icon_color=icon_color),
        popup= popup
    )
    marker_cluster.add_child(marker)
site_map

### Lets calculate the distance between a launch site and its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [11]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

### Calculate the distance from launch site to coastline

In [12]:
from math import sin, cos, sqrt, atan2, radians

# function to calculate the distance when given latitude and longitude coordenates
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [13]:
# group the Launch Site, Lat and Long 
grouped = spacex_df.groupby(['Launch Site'], as_index=False).first()
space_cor = grouped[['Launch Site', 'Lat', 'Long']]

coastal_long = [-80.56778,-80.56793,-80.5711,-120.62641]

# add coastal longitude coordinates, coastal latitude and launch line latitude are the same.
for index, long in enumerate(zip(coastal_long)):
    space_cor.loc[index, 'coastal_long'] = long

space_cor

C:\Users\steph\AppData\Local\Temp\ipykernel_12600\2547147011.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  space_cor.loc[index, 'coastal_long'] = long


,Launch Site,Lat,Long,coastal_long
0,CCAFS LC-40,28.562302,-80.577356,-80.56778
1,CCAFS SLC-40,28.563197,-80.576820,-80.56793
2,KSC LC-39A,28.573255,-80.646895,-80.57110
3,VAFB SLC-4E,34.632834,-120.610745,-120.62641


In [14]:
for _, row in space_cor.iterrows():
    distance_coastline = calculate_distance(row['Lat'], row['Long'], row['Lat'], row['coastal_long'])
    print(f"The distance from {row['Launch Site']} is: {distance_coastline:.2f}(km)")

The distance from CCAFS LC-40 is: 0.94(km)
The distance from CCAFS SLC-40 is: 0.87(km)
The distance from KSC LC-39A is: 7.40(km)
The distance from VAFB SLC-4E is: 1.43(km)


### Create and add a folium.Marker on your selected closest coastline point on the map

In [15]:
# As shown in above the closest Launch Site to the coastal line is CCAFS SLC-40
coordinate = [28.563197, -80.56793]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(0.87),
       )
   )
site_map.add_child(distance_marker)

## Add a line from launch site to coastal line

In [16]:
coordinates = [[28.563197,-80.576820],[28.563197,-80.56793]]
line= folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(line)

site_map

### Calculate the distance from CCAFS SLC-40 to closest city. Add Marker and line

In [17]:
distance_city = calculate_distance(28.563197,-80.576820,28.0836,-80.6081)
print(f"{distance_city:.2f}")

53.43


In [18]:
coordinate = [[28.563197,-80.576820],[28.1059,-80.6369]]
coordinates = [28.1059,-80.6369]
distance_marker = folium.Marker(
   coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(53.43),
       )
   )
line= folium.PolyLine(locations=coordinate, weight=1, color='blue')

site_map.add_child(line)
site_map.add_child(distance_marker)

site_map

### Calculate the distance from CCAFS SLC-40 to closest railroad. Add Marker and line

In [19]:
distance_city = calculate_distance(28.563197,-80.576820,28.57207,-80.58528)
print(f"{distance_city:.2f}")

1.29


In [20]:
coordinate = [[28.563197,-80.576820],[28.57207,-80.58528]]
distance_marker = folium.Marker(
   coordinate[1],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(1.29),
       )
   )
line= folium.PolyLine(locations=coordinate, weight=1, color='blue')

site_map.add_child(line)
site_map.add_child(distance_marker)

site_map

### Calculate the distance from CCAFS SLC-40 to closest Highway. Add Marker and line

In [21]:
distance_city = calculate_distance(28.563197,-80.576820,28.56347,-80.57079)
print(f"{distance_city:.2f}")

0.59


In [22]:
coordinate = [[28.563197,-80.576820],[28.56347,-80.57079]]
distance_marker = folium.Marker(
   coordinate[1],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(0.59),
       )
   )
line= folium.PolyLine(locations=coordinate, weight=1, color='black')

site_map.add_child(line)
site_map.add_child(distance_marker)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
    - No. The distance is approximate 1.29 km and its a NASA railroad where the Space X and other spacial projects use to transport large cargo. 
*   Are launch sites in close proximity to highways?
    - I would say yes with a distance approximate of 0.59 km. The reason why is because we dont know if that highway is only use to get to the launch site or for other reason. 
*   Are launch sites in close proximity to coastline?
    - Yes with a distance of 0.87km
*   Do launch sites keep certain distance away from cities?
    - Yes with a distance of 53 miles